# Hierarchical Modeling in PyMC3

## Introduction

Remixed from <https://docs.pymc.io/notebooks/multilevel_modeling.html>.

My goals for this tutorial are to:

- (Re)Introduce hierarchical modeling
- Demo PyMC3
- Convince you of the flexibility of statistical modeling in the Bayesian framework
- Give you some general tools for multi-level modeling

## Setup

Install all of the necessary packages.

E.g.

```bash
conda create -n bhm --file requirements.txt
```

Run the following to get the repository and data

```bash
git clone https://github.com/bsmith89/pymc3_hierarchical_tutorial
cd pymc3_hierarchical_tutorial
conda activate bhm
make data/clean_data.tsv
```

## Load and get familiar with the data

Now we'll load this cleaned dataset.

In [ ]:
import pandas as pd

data = pd.read_table('data/clean_data.tsv')
data.shape

In [ ]:
data.head()

Plot the distribution of radon measurements.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(data.radon, bins=100)
plt.yscale('symlog')

Clearly a skewed distribution, and bounded at 0.

We would normally think to log-transform these data, but we have a bunch of
observations that were below the detection limit (0.1 of whatever units these measurements are in).

In [ ]:
data[data.radon != 0].radon.min()

### Data transformations

We'll replace those zeros with half of the detection limit, purely as a heuristic.

In [ ]:
data['radon_nonzero'] = data.radon
data.radon_nonzero[data.radon == 0] = 0.05

Now we can see that these data are approximately normally distributed.

Much better for our purposes.

In [ ]:
import numpy as np

_ = plt.hist(data.radon_nonzero.apply(np.log), bins=100)

In [ ]:
data['log_radon'] = data['radon_nonzero'].apply(np.log)

To make our analysis easier to manage, we'll start by just looking at measurements from Minnesota, only.

## Predicting radon measurements in MN

In [ ]:
d = data[data.state == 'MN']

The first factor that we're going to consider for predicting radon levels is
where the measurement was taken.

We know that radon comes out of the ground, so it makes sense that levels are
higher in basements compared to the rest of a house.

That's approximately what we see.

In [ ]:
plt.scatter('is_basement', 'log_radon', data=d, alpha=0.5)

In [ ]:
import seaborn as sns

sns.violinplot('is_basement', 'log_radon', data=d)

### Traditional linear regression

#### Classic linear model (Complete pooling)

Now we can start by taking a standard, linear regression approach to analyzing these data.

Since we expect measurements taken in the basement to be higher, we can start by
modeling this using OLS and the following formula.

In [ ]:
import patsy

y, x = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
n, r = x.shape

x.head()

I'm using the package `patsy` to build my design matrices, because it
gives me convenient "R-like" formulas.

In [ ]:
import statsmodels.api as sm

fit0 = sm.OLS(y, x).fit()

fit0.summary()

This linear regression shows us that, indeed, measurements
taken in the basement are higher than in other parts of the house.

With our relatively large sample size, our p-value is VERY significant,
even though the R^2 is quite small.

It's worth doing a few standard diagnostics to make sure this model is
appropriate.

In [ ]:
def jitter(x, perc=0.05):
    span = max(x) - min(x)
    return x + (np.random.rand(len(x)) - 0.5) * perc * span

In [ ]:
plt.scatter(jitter(fit0.predict(), 0.1),
            fit0.resid_pearson, s=0.5)

In [ ]:
resid_df = pd.DataFrame({'is_basement': d.is_basement, 'resid': fit0.resid_pearson})

sns.violinplot('is_basement', 'resid', data=resid_df)

In [ ]:
plt.scatter(jitter(d['is_basement']), d['log_radon'], s=1)
plt.plot([0, 1], [fit0.params[0], fit0.params[0] + fit0.params[1]], color='k')

As you can see, this model fits the data.

However, we know that the basic assumptions of this linear regression (**I**ID) is violated.

Since these data are geographically distributed, i.e. by county, they are not independent.
Some regions may have higher or lower radon levels on average.
This is consistent with the fact that we know radon is a product of uranium decay, and
uranium levels vary across Minnesota.

#### Linear model with county effects.

In [ ]:
y, x = patsy.dmatrices('log_radon ~ 0 + state_county + is_basement', data=d, return_type='dataframe')
n, r = x.shape

fit1 = sm.OLS(y, x).fit()
fit1.summary()

In [ ]:
plt.scatter(fit1.predict(),
            y, s=0.5)
plt.plot([0, 3], [0, 3])

That works pretty well!

There are, however, two problems with this model.

The biggest is that the estimates for county effects are basically independent,
and reflect a simple county mean.
This is problematic, because some counties might have very few observations,
and there coefficients will not be a good estimate for the county-level expectation.

In [ ]:
d.groupby('state_county').apply(len).sort_values()

We're going to solve this problem using a hierarchical model, and we're going
to build our model under a Bayesian framework.

### Bayesian clone of OLS

But first I need to introduce PyMC3 and show how to use it.

I'll therefore start by building the above models with PyMC3.

#### Complete pooling model

In [ ]:
import pymc3 as pm
import theano.tensor as tt

with pm.Model() as model_classic:
    beta = pm.Normal('beta', sd=10, shape=(r, 1))
    sigma = pm.HalfCauchy('sigma', beta=2)
    
    mu = tt.dot(x.values, beta)
    
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
model_classic.logp(model_classic.test_point)

In [ ]:
with model_classic:
    trace_classic = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace_classic)

In [ ]:
with model_classic:
    ppc_classic = pd.DataFrame(pm.sample_posterior_predictive(trace_classic, samples=1)['obs'][0],
                        index=y.index, columns=y.columns)
    
plt.scatter(ppc_classic.log_radon, y.log_radon, s=2)
plt.plot([-1, 4], [-1, 4])

#### Model flexibility - substituting in the t-distribution

In [ ]:
with pm.Model() as model0:
    beta = pm.Normal('beta', sd=10, shape=(r, 1))
    
    mu = tt.dot(x.values, beta)
    
    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
model0.logp(model0.test_point)

In [ ]:
with model0:
    trace0 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace0)

In [ ]:
with model0:
    ppc0 = pd.DataFrame(pm.sample_posterior_predictive(trace0, samples=1)['obs'][0],
                        index=y.index, columns=y.columns)
    
plt.scatter(ppc0.log_radon, y.log_radon, s=2)
plt.plot([-1, 4], [-1, 4])

In [ ]:
models = [
                     ('complete_pooling_normal', model_classic, trace_classic),
                     ('complete_pooling', model0, trace0),
         ]



model_compare = pm.compare({x[1]: x[2] for x in models}, ic='LOO')
model_compare.rename(index={i: x[0] for i, x in enumerate(models)}, inplace=True)
model_compare

#### No pooling model

In [ ]:
counties = d.state_county.unique()
county_lookup = dict(zip(counties, range(len(counties))))
d['county_idx'] = d.state_county.replace(county_lookup)

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]


with pm.Model() as model1:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    gamma = pm.Normal('gamma', sd=10, shape=(r2, 1))
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)
    
    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model1:
    trace1 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace1, var_names=['beta', 'sigma', 'nu'])

In [ ]:
pm.forestplot(trace1, var_names=['gamma'])

### Partial pooling

#### Random effects with partial pooling

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]


with pm.Model() as model2:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_mean = pm.Normal('gamma_hyper_mean', sd=10)
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma = pm.Normal('gamma', mu=gamma_hyper_mean, sd=gamma_hyper_sd, shape=(r2, 1))
        
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)
    
    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model2:
    trace2 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace2, var_names=['beta', 'gamma_hyper_mean', 'gamma_hyper_sd', 'sigma', 'nu'])

#### Reparameterize partial pooling model (Noncentered reparameterization)

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]


with pm.Model() as model3:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_mean = pm.Normal('gamma_hyper_mean', sd=10)
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mean + gamma_hyper_sd * gamma_)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)
    
    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model3:
    trace3 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace3, var_names=['beta', 'gamma_hyper_mean', 'gamma_hyper_sd', 'sigma', 'nu'])

#### Add county-level uranium as predictor

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]


assert np.all(u.index == x2.columns)


with pm.Model() as model4:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)

    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model4:
    trace4 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace4,
             var_names=['beta', 'gamma_hyper_beta0',
                        'gamma_hyper_beta', 'gamma_hyper_sd',
                        'sigma', 'nu'])

In [ ]:
pm.forestplot(trace4, var_names=['gamma'])

In [ ]:
trace4.gamma_hyper_mu.T[0]

In [ ]:
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
gamma_dist_prd = pd.DataFrame(np.quantile(trace4.gamma_hyper_mu, quantiles, axis=0).T[0],
                              columns=quantiles, index=counties)
gamma_dist_obs = pd.DataFrame(np.quantile(trace4.gamma, quantiles, axis=0).T[0],
                              columns=quantiles, index=counties)

In [ ]:
plt.scatter(gamma_dist_prd[0.5], gamma_dist_obs[0.5])
plt.vlines(gamma_dist_prd[0.5], gamma_dist_obs[0.25], gamma_dist_obs[0.75], lw=0.5)
plt.hlines(gamma_dist_obs[0.5], gamma_dist_prd[0.25], gamma_dist_prd[0.75], lw=0.5)

### County-level random slope

#### Random slope without pooling

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]

x1x2 = pd.DataFrame(x1.values * x2.values,
                    index=x1.index,
                    columns=x2.columns + ':' + 'is_basement[T.True]')

assert np.all(u.index == x2.columns)


with pm.Model() as model5:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    kappa = pm.Normal('kappa', sd=10, shape=(r2, 1))
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma) + tt.dot(x1x2.values, kappa)

    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model5:
    trace5 = pm.sample(tune=2000)

In [ ]:
pm.traceplot(trace5, var_names=['beta',
                                'gamma_hyper_beta', 'gamma_hyper_sd',
                                'sigma', 'nu'])

In [ ]:
pm.forestplot(trace5, var_names=['kappa'])

In [ ]:
i = -200

mu_expect5 = mu.eval({getattr(model5, k.name): trace5[i][k.name]
                      for k in model5.vars
                      if k.name not in ['sigma_log__', 'nu_log__']})
plt.scatter(mu_expect5, y.values, s=2)
plt.plot([0, 2], [0, 2], color='k')

In [ ]:
plt.scatter(trace5['gamma_hyper_beta'][:,0,0],
            trace5['gamma_hyper_beta'][:,1,0])

#### Random slope with partial pooling

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]

x1x2 = pd.DataFrame(x1.values * x2.values,
                    index=x1.index,
                    columns=x2.columns + ':' + 'is_basement[T.True]')

assert np.all(u.index == x2.columns)

with pm.Model() as model6:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    kappa_hyper_sd = pm.HalfCauchy('kappa_hyper_sd', beta=2)
    kappa_ = pm.Normal('kappa_', shape=(r2, 1))
    kappa = pm.Deterministic('kappa', kappa_ * kappa_hyper_sd)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma) + tt.dot(x1x2.values, kappa)

    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model6:
    trace6 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace6, var_names=['beta',
                                'gamma_hyper_beta', 'gamma_hyper_sd',
                                'kappa_hyper_sd', 'nu'])

In [ ]:
pm.forestplot(trace6, var_names=['kappa'])

#### Random slope with a predictor

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]

u_ni = patsy.dmatrix('county_uranium - 1', data=_u, return_type='dataframe')
r4 = u_ni.shape[1]

x1x2 = pd.DataFrame(x1.values * x2.values,
                    index=x1.index,
                    columns=x2.columns + ':' + 'is_basement[T.True]')

assert np.all(u.index == x2.columns)

with pm.Model() as model7:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    kappa_hyper_beta = pm.Normal('kappa_hyper_beta', sd=10, shape=(r4, 1))
    kappa_hyper_mu = pm.Deterministic('kappa_hyper_mu', tt.dot(u_ni.values, kappa_hyper_beta))
    kappa_hyper_sd = pm.HalfCauchy('kappa_hyper_sd', beta=2)
    kappa_ = pm.Normal('kappa_', shape=(r2, 1))
    kappa = pm.Deterministic('kappa', kappa_hyper_mu + kappa_ * kappa_hyper_sd)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma) + tt.dot(x1x2.values, kappa)

    sigma = pm.HalfCauchy('sigma', beta=2)
    nu = pm.HalfCauchy('nu', beta=2)
    obs = pm.StudentT('obs', mu=mu, sigma=sigma, nu=nu, observed=y)

In [ ]:
with model7:
    trace7 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace7, var_names=['beta',
                                'gamma_hyper_beta', 'gamma_hyper_sd',
                                'kappa_hyper_beta', 'kappa_hyper_sd',
                                'sigma', 'nu'])

In [ ]:
pm.forestplot(trace7, var_names=['kappa'])

In [ ]:
with model7:
    ppc7 = pd.DataFrame(pm.sample_posterior_predictive(trace7, samples=1)['obs'][0],
                        index=y.index, columns=y.columns)
    
plt.scatter(ppc7.log_radon, y.log_radon, s=2)
plt.plot([-1, 4], [-1, 4])

### Model Comparison

In [ ]:
models = [
                     ('complete_pooling_normal', model_classic, trace_classic),
                     ('complete_pooling', model0, trace0),
                     ('no_pooling', model1, trace1),
                     ('partial_pooling', model2, trace2),
                     ('partial_pooling_reparam', model3, trace3),
                     ('county_uranium', model4, trace4),
                     ('county_uranium_no_pooling_slope', model5, trace5),
                     ('county_uranium_partial_pooling_slope', model6, trace6),
                     ('county_uranium_both', model7, trace7),
         ]



model_compare = pm.compare({x[1]: x[2] for x in models}, ic='WAIC')
model_compare.rename(index={i: x[0] for i, x in enumerate(models)}, inplace=True)
pm.compareplot(model_compare, figsize=(15, 5))
plt.legend()

model_compare